# import library

In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

import csv
import pandas as pd

import itertools

import statsmodels
import statsmodels.tsa.arima.model

from datetime import date
from datetime import timedelta

from webdriver_manager.chrome import ChromeDriverManager

# Scraping data

In [5]:
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.cursbnr.ro/curs-valutar-bnr')
select = Select(browser.find_element( By.NAME ,'currency'))
nr_student = 42
nr_valut = len(select.options)

if (nr_student >= nr_valut):
    while(nr_student >= nr_valut):
        nr_student = nr_student-nr_valut
else:
    nr_student= nr_student -1
    
select.select_by_index(nr_student)
startdate = browser.find_element_by_name('dataStart')
startdate.clear()
startdate.send_keys('27/03/2005')
browser.find_element_by_id('formbuton').click()
html = browser.page_source
soup = BeautifulSoup(html.lower(), "html.parser")
lista_td = soup.find( attrs={'class':'table table-md table-striped text-center'}).find('tbody').find_all('td')
lista_dict = []
row_dict = {}
for i in range(len(lista_td)):

    if i%2 == 0:
        row_dict = {}
        row_dict['data'] = lista_td[i].text
    if i%2 != 0:
        row_dict['curs vs RON'] = lista_td[i].text
        lista_dict.append(row_dict)
    
df = pd.DataFrame(lista_dict)
df.to_csv(r'D:\my_data.csv', index=False)
browser.close()

[WDM] - Downloading:  94%|████████████████████████████████████████████████████▊   | 6.10M/6.46M [00:01<00:00, 6.18MB/s]<ipython-input-5-88074e25a1fc>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-5-88074e25a1fc>:14: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  startdate = browser.find_element_by_name('dataStart')
<ipython-input-5-88074e25a1fc>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_id('formbuton').click()
[WDM] - Downloading: 100%|█████████████████████████████████████████████████████████| 6.46M/6.46M [00:24<00:00, 276kB/s]


# ARIMA model

In [7]:

new_df = pd.read_csv("D:\my_data.csv")
new_df['data'] = pd.to_datetime(new_df['data'], format="%d.%m.%Y")
new_df = new_df.set_index('data').asfreq('D')
new_df.curs=new_df['curs vs RON'].fillna(method="ffill")
y = new_df['curs vs RON']
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
mod = statsmodels.tsa.arima.model.ARIMA(y,
                       order=pdq[1],
                        enforce_stationarity=False,
                        enforce_invertibility=False)
results = mod.fit()
results.aic
dict_list = []

for param in pdq:
   #for param_seasonal in seasonal_pdq:
    try:
            mod = statsmodels.tsa.arima.model.ARIMA(y,
            order=pdq[1],
            enforce_stationarity=False,
            enforce_invertibility=False)
            results = mod.fit()
            
            # lista cu dictionare de valori; aceleasi key inseamna ca la final vom putea transforma lista in dataframe
            dict_list.append({'pdq':param,
                              #'seasonal_pdq':param_seasonal,
                             'AIC':results.aic })
            
            #print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
    except:
            continue
            
best_param = pd.DataFrame(dict_list).sort_values(by = 'AIC', ascending = True).pdq[0]
mod = statsmodels.tsa.arima.model.ARIMA(y,
                                order=best_param,
                                
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()
try:
    print(results.summary().tables[1])
    print("Prediction next day:")
    print(results.predict(date.today()+timedelta(days=1)))
except:
    print("Display problems")

<ipython-input-7-561227caba2c>:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  new_df.curs=new_df['curs vs RON'].fillna(method="ffill")


                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.2830      0.006    856.400      0.000       5.271       5.295
sigma2         0.1664      0.004     43.866      0.000       0.159       0.174
Prediction next day:
2022-10-30    5.282952
Freq: D, dtype: float64
